# **성능평가**
학습이 완료되면 모델의 성능을 평가하여 모델이 좋은지 나쁜지 확인해야 한다. 모델의 성능을 평가하기 위한 기준은 몇 가지가 있다.

### **정확도(Accuracy)**
정확도(accuracy)는 가장 많이 사용되는 성능 평가 기준이다.

`정확도 = (올바르게 분류한 샘플 수) / (전체 샘플 수)`

정확도는 가장 많이 사용되지만 절대적인 것은 아니며, 한 클래스가 다른 클래스에 비해 항상 월등하게 많다면 문제가 된다.

### **혼동 행렬(Confusion Matrix)**
혼동 행렬(Confusion Matrix)은 모델이 예측을 하면서 얼마나 혼동하고 있는지를 나타내는 행렬이다.



![nn](outputs/4-01-1.png)

혼동 행렬을 알기 위해 필요한 네 가지 개념이 있다.
- TP(True Positive) - 긍정을 긍정으로 올바르게 예측한 것
- FN(False Negative) - 긍정을 부정으로 잘못 예측한 것
- FP(False Positive) - 부정을 긍정으로 잘못 예측한 것
- TN(True Negative) - 부정을 부정으로 올바르게 예측한 것

혼동 행렬 표로부터 모델의 성능을 측정하는 4가지 지표가 있다.
1. `정확도(Accuracy) = (TP + TN) / (TP + FN + FP + TN)`
2. `재현도(Recall) = (TP) / (TP + FN)`
3. `정밀도(Precision) = (TP) / (TP + FP)`
4. `F1 score = 2 / {(1 / Recall) + (1 / Precision)} = 2 * (Recall * Precision) / (Recall + Precision)`

예를 들어, 관심 범주가 질병이 있는 환자라면, 정확도는 전체 중에서 환자를 환자로, 정상인을 정상인으로 진단한 비율이다. 

재현도는 실제 질병이 있는 사람을 환자라고 진단한 비율이다. 정밀도는 질병이 있다고 진단한 사람 중 실제 환자의 비율이다. 

F1 score는 재현도와 정밀도의 성능을 동시에 고려하기 위해 사용하는 지표이다. 재현도와 정밀도의 조화 평균이며, 0과 1 사이의 값을 가지며 1에 가까울수록 성능이 좋음을 나타낸다.

우리가 사용할 지표는 F1 score인데, 이 지표는 클래스 간 데이터의 불균형이 심할 때 유용하다. 우리가 사용할 데이터는 클래스 간 불균형이 심하기 때문에 이 지표를 사용한다.

다음 예시를 통해 Recall과 Precision을 이용해서 F1 score를 계산해보고, 이를 f1_score 함수로 계산한 값과 비교해보자.

Recall, Precision, F1 score를 계산하는 함수를 사용하기 위해 sckit-learn 모듈의 metrics 클래스에서 recall_score, precision_score, f1_score 함수를 import한다.

In [1]:
from sklearn.metrics import (
	recall_score,
	precision_score,
	f1_score
)

In [2]:
y_true = [0, 1, 1, 0, 1, 0, 0, 1, 0]
y_predict = [0, 1, 1, 0, 1, 1, 0, 1, 1]

rec_score = recall_score(y_true, y_predict)
print("recall score:", rec_score)
prec_score = precision_score(y_true, y_predict)
print("precision score:", prec_score)

my_f1_score = 2 * (rec_score * prec_score) / (rec_score + prec_score)
print("my_f1_score:", my_f1_score)
print("f1_score:",f1_score(y_true, y_predict))

recall score: 1.0
precision score: 0.6666666666666666
my_f1_score: 0.8
f1_score: 0.8


## **Train/Test 관련**
모델의 성능을 평가하기 위해 데이터를 두 부분으로 나눠야 한다. 

데이터의 일부는 모델의 학습에 사용되고 이를 훈련(train) 데이터라고 한다. 데이터의 나머지 부분은 훈련된 모델의 성능을 평가하는 데 사용되고 이를 테스트(test) 데이터라고 한다.

이렇게 데이터를 두 부분으로 나눠서 훈련에 사용되지 않은 데이터로 평가를 하는 이유는 모델이 과적합이 되지 않고 일반화 되었는지 평가하기 위함이다. 

과적합(overfitting)이란 훈련 데이터에 너무 많이 학습되어 훈련 데이터에 대해서는 높은 성능을 보이지만, 새로운 데이터에 대해서는 성능이 저하되는 현상을 의미한다. 

즉, 훈련 데이터에 대해서 과하게 학습되지 않고 일반화된 성능을 평가하기 위해 데이터를 두 부분으로 나누어 사용한다.

## **Cross Validation**
교차 검증(Cross Validation)이란 모델을 학습시킬 때 훈련 데이터를 또 다시 훈련 데이터와 검증(validation) 데이터로 나누어 훈련 데이터로 학습한 후, 검증 데이터를 통해 모델의 성능을 평가하는 방법이다.

교차 검증의 장점은 특정 데이터에 대한 과적합을 방지하고, 일반화된 모델을 만들 수 있게 해주는 것 등이 있다.

교차 검증의 단점은 모델에 대한 훈련 및 평가에 소요되는 시간이 증가한다는 점이다.

### **K-fold Cross Validation**
K-fold Cross Validation은 모델의 성능을 평가하는 기법 중 하나로, 데이터를 K개의 fold로 나누고 K번의 실험을 통해 모델의 성능을 평가한다.

각 실험에서 하나의 fold를 테스트 데이터로 사용하고, 나머지 K-1개의 fold를 훈련 데이터로 사용한다.

테스트 데이터로 사용할 fold를 바꿔가며 총 K번 실험을 진행하여 얻은 성능 지표의 평균을 최종 성능 지표로 사용한다.

K-fold Cross Validation을 사용하면 데이터가 적더라도 신뢰성을 확보할 수 있다.



다음은 K-fold Cross Validation을 하는 예시이다.

K-fold CV의 실습을 하기 전에 여기서 사용할 데이터에 대해 알아보자.

이번 예시에서 사용할 데이터는 tox21 데이터이다.

tox21 데이터는 총 166개의 속성(maccs_1, ..., maccs166)과 1개의 라벨(NR-AR)으로 구성되어 있다.

In [3]:
# Import

# 표 형식의 데이터 등 다양한 데이터를 쉽게 다룰 수 있도록(데이터 결합 등) 해주는 모듈
import pandas as pd

# 다차원 배열을 쉽게 처리하고 효율적으로 사용할 수 있도록 해주는 모듈
import numpy as np

# 범주형 변수 예측에 사용되는 모델 중 하나
# 분류 문제(0인지 1인지 분류처럼)에 사용되는 기계학습 알고리즘 중 하나
# 현재 예시에서 사용할 모델
from sklearn.linear_model import LogisticRegression

# scikit-learn에서 제공하는 K-Fold
from sklearn.model_selection import KFold

### **Data 불러오기**

In [4]:
# 사용할 데이터가 들어있는 폴더
folder = '../0.Data'
# 사용할 데이터의 파일명
train_data_file = 'tox21_train.csv'
# ../0.Data/tox21_train.csv과 같이 파일 경로로 만듦
train_file_path = f'{folder}/{train_data_file}'

# 파일 경로에 해당하는 csv 파일 불러오기
# csv 파일이란 comma-separated values로, 쉼표로 구분된 데이터 파일임
df = pd.read_csv(train_file_path)
print(df.shape)

(11130, 167)


In [5]:
# data에서 feature 부분과 label 부분 분리
X = df.iloc[:, :166]
y = df.iloc[:, 166]

In [6]:
# ================My K-fold================

# k-fold에서 데이터를 몇 개의 fold로 나눌 지 정함
k = 5

# feature의 데이터 개수를 저장
size = len(X)

# feature를 k개의 fold로 나누기 위해 몇 개의 데이터 단위로 끊어야 하는 지 계산
fold_size = (int)(size / k)

# 각 실험에 대한 accuracy를 저장하기 위한 배열
accuracies = np.empty(k)

# fold 개수만큼 실험 반복
for i in range(k):
	# [arange 함수 참고]
	# numpy 모듈의 arange 함수는 인자로 들어온 범위만큼의 등차수열 배열을 반환
	# 사용 -> np.arange(start, end, step) (start와 step을 지정해주지 않으면 각각 0, 1이 기본값으로 적용)
	# ex. np.arange(0, 10, 1) -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
	# ex. np.arange(10) -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

	# [concatenate 함수 참고]
	# numpy 모듈의 concatenate 함수는 인자로 들어온 배열을 합쳐서 새로운 배열을 반환
	# ex. np.concatenate((np.array([1, 2, 3]), np.array([4, 5, 6]))) -> [1, 2, 3, 4, 5, 6]

	# 분할할 train data & validation data의 index 저장
		# feature 중에서 validation data로 사용할 부분의 index 저장
		# 즉, validation data로 사용할 fold의 index를 저장
	val_index = np.arange(i * fold_size, (i + 1) * fold_size)
		# validation data로 사용할 부분을 제외한 부분의 index를 합쳐서 저장
		# train data로 사용할 부분의 index임
	train_index = np.concatenate((np.arange(i * fold_size), np.arange((i + 1) * fold_size, size)))

	# train data & validation data 생성
	# X 중에서 위에서 지정한 index에 해당하는 값을 train과 validation에 나누어 저장
	X_train = X.iloc[train_index]
	X_val = X.iloc[val_index]
	y_train = y.iloc[train_index]
	y_val = y.iloc[val_index]

	# 사용할 모델 지정(Logistic Regression 모델 사용)
	model = LogisticRegression()

	# 모델에 학습
	model.fit(X_train, y_train)

	# 매 실험마다 계산한 정확도를 accuracies 배열에 하나씩 저장
	# feature_test의 test data에 대해 위에서 학습한 모델이 추측한 결과와 실제 정답인 label_test를 비교하여 accuracy 계산
	accuracies[i] = model.score(X_val, y_val)

# 위에서 계산한 전체 accuracy 출력
print("My K-fold Accuracies:", accuracies)

# 모든 accuracy의 평균을 계산하여 최종 accuracy 출력
print("My K-fold Mean Accuracy:", accuracies.mean())

My K-fold Accuracies: [0.82659479 0.82749326 0.89218329 0.84141959 0.83468104]
My K-fold Mean Accuracy: 0.8444743935309973


In [7]:
# ================Scikit K-fold================

In [8]:
# [방법 1]

# k-fold에서 데이터를 몇 개의 fold로 나눌 지 정함
k = 5

# k개의 fold로 cross validation하는 K-Fold 객체 생성
kfold = KFold(n_splits = k)

# 사용할 모델 지정(Logistic Regression 모델 사용)
model = LogisticRegression()

# 각 실험에 대한 accuracy를 저장하기 위한 배열
accuracies = np.empty(k)

# k번 만큼 실험을 반복하며, 매 회차마다의 train 데이터를 다시 train과 validation data 범위의 index를 각각 train_index와 val_index에 저장
for train_index, val_index in kfold.split(X, y):
	# validation index 범위 출력해서 확인
	print(val_index)
	
	# train data & validation data 생성
	# X 중에서 위에서 지정한 index에 해당하는 값을 train과 val 나누어 저장
	X_train, y_train = X.iloc[train_index], y.iloc[train_index]
	X_val, y_val = X.iloc[val_index], y.iloc[val_index]

	# 모델에 학습
	model.fit(X_train, y_train)

	# 매 실험마다 계산한 정확도를 accuracies 배열에 하나씩 저장
	# X_val의 validation data에 대해 위에서 학습한 모델이 추측한 결과와 실제 정답인 y_val를 비교하여 accuracy 계산
	accuracies[i] = model.score(X_val, y_val)


# 위에서 계산한 전체 accuracy 출력
print("Scikit-learn K-fold Accuracies:", accuracies)

# 모든 accuracy의 평균을 계산하여 최종 accuracy 출력
print("Scikit-learn K-fold Mean Accuracy:", accuracies.mean())

[   0    1    2 ... 2223 2224 2225]
[2226 2227 2228 ... 4449 4450 4451]
[4452 4453 4454 ... 6675 6676 6677]
[6678 6679 6680 ... 8901 8902 8903]
[ 8904  8905  8906 ... 11127 11128 11129]
Scikit-learn K-fold Accuracies: [0.82659479 0.82749326 0.89218329 0.84141959 0.83468104]
Scikit-learn K-fold Mean Accuracy: 0.8444743935309973


In [9]:
# [방법 2]

# cross validation에서 score를 계산하는 함수
from sklearn.model_selection import cross_val_score

# k-fold에서 데이터를 몇 개의 fold로 나눌 지 정함
k = 5

# k개의 fold로 cross validation하는 K-Fold 객체 생성
kfold = KFold(n_splits = k)

# 사용할 모델 지정(Logistic Regression 모델 사용)
model = LogisticRegression()

# 사용할 model, X, y, 사용할 cross validation 객체로 cross validation 했을 때의 accuracy 배열을 results에 저장
results = cross_val_score(model, X, y, cv = kfold)

# 전체 accuracy 출력
print("Scikit-Learn K-fold Accuracies:", results)

# 모든 accuracy의 평균을 계산하여 최종 accuracy 출력
print("Scikit-Learn K-fold Mean Accuracy:", results.mean())

Scikit-Learn K-fold Accuracies: [0.82659479 0.82749326 0.89218329 0.84141959 0.83468104]
Scikit-Learn K-fold Mean Accuracy: 0.8444743935309973


위와 같이 K-fold CV를 직접 구현한 결과와 Scikit-Learn의 K-fold를 사용한 결과 모두 동일함을 알 수 있다.


하지만, 데이터가 불균형한 경우 K-fold는 효과적이지 않을 수 있다.

예를 들어, true가 100000개 있고 false가 50개라고 했을 때, K-fold를 적용하게 된다면 false가 없는 부분만 학습을 하고 false를 분류할 수 없는 상황이 발생할 수 있다.

이와 같이, 데이터가 불균형한 경우에 사용할 수 있는 방법이 Stratified K-fold Cross Validation이다.

이러한 K-fold Cross Validation의 문제를 해결하기 위해 Stratified K-fold Cross Validation을 배워보자.

### **Stratified K-fold Cross Validation**

Stratified K-fold Cross Validation이란 K-fold CV의 target의 비율을 균일하게 유지 못한다는 문제점을 해결하여 target의 비율을 균일하게 해주는 방식이다.

우리가 추후 사용할 데이터는 불균형 데이터이기 때문에 Stratified K-fold Cross Validation을 사용할 예정이다.

In [10]:
# Stratified K-Fold
from sklearn.model_selection import StratifiedKFold

# k-fold에서 데이터를 몇 개의 fold로 나눌 지 정함
k = 5

# k개의 fold로 cross validation하는 Stratified K-Fold 객체 생성
strkfold = StratifiedKFold(n_splits = k)

# 사용할 모델 지정(Logistic Regression 모델 사용)
model = LogisticRegression(max_iter=1000)

# 각 실험에 대한 accuracy를 저장하기 위한 배열
accuracies = np.empty(k)

# 반복문이 돌아가면서 accuracies 배열에 값을 저장하기 위한 index 변수
i = 0

# k번 만큼 실험을 반복하며, 매 회차마다의 train과 validation data 범위의 index를 각각 train_index와 val_index에 저장
for train_index, val_index in strkfold.split(X, y):
	# val index 범위 출력해서 확인
	print(val_index)
	
	# test data & train data 생성
	# X 중에서 위에서 지정한 index에 해당하는 값을 train과 test에 나누어 저장
	X_train, X_val = X.iloc[train_index], X.iloc[val_index]
	y_train, y_val = y.iloc[train_index], y.iloc[val_index]

	# 모델에 학습
	model.fit(X_train, y_train)

	# 매 실험마다 계산한 정확도를 accuracies 배열에 하나씩 저장
	# X_test의 test data에 대해 위에서 학습한 모델이 추측한 결과와 실제 정답인 y_test를 비교하여 accuracy 계산
	accuracies[i] = model.score(X_val, y_val)
	
	i = i + 1


# 위에서 계산한 전체 accuracy 출력
print("Stratified K-fold Accuracies:", accuracies)

# 모든 accuracy의 평균을 계산하여 최종 accuracy 출력
print("Stratified K-fold Mean Accuracy:", accuracies.mean())

[   0    1    2 ... 6675 6676 6677]
[1168 1169 1170 ... 7788 7789 7790]
[2333 2334 2335 ... 8901 8902 8903]
[ 3490  3491  3493 ... 10014 10015 10016]
[ 4651  4652  4653 ... 11127 11128 11129]
Stratified K-fold Accuracies: [0.88140162 0.88005391 0.89892183 0.89308176 0.88993711]
Stratified K-fold Mean Accuracy: 0.8886792452830189


위 결과와 같이 Stratified K-fold CV 결과의 분산이 일반 K-fold CV 결과의 분산보다 더 작고 성능이 더 좋게 나옴을 알 수 있다.

# **Evaluation 시작**

## **Import**

In [11]:
import numpy as np

# 표 형식의 데이터 등 다양한 데이터를 쉽게 다룰 수 있도록(데이터 결합 등) 해주는 모듈
import pandas as pd

# Stratified K-Fold
from sklearn.model_selection import StratifiedKFold

# 첫 번째 모델
from sklearn.linear_model import LogisticRegression

# 두 번째 모델
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import (
	accuracy_score,
	roc_auc_score,
	recall_score,
	precision_score,
	f1_score
)

from tqdm import tqdm

from itertools import product
from collections.abc import Iterable

import warnings
warnings.filterwarnings('ignore')

## **0. Data 불러오기**

In [12]:
# pandas dataframe의 모든 열을 출력하도록 설정
pd.set_option('display.max_columns', None)

# train과 test csv 파일 불러오기
# csv 파일이란 comma-separated values로, 쉼표로 구분된 데이터 파일임
df = pd.read_csv('../0.Data/tox21_train.csv')
df_test = pd.read_csv('../0.Data/tox21_test.csv')

In [13]:
# df 출력
df

,maccs_1,maccs_2,maccs_3,maccs_4,maccs_5,maccs_6,maccs_7,maccs_8,maccs_9,maccs_10,maccs_11,maccs_12,maccs_13,maccs_14,maccs_15,maccs_16,maccs_17,maccs_18,maccs_19,maccs_20,maccs_21,maccs_22,maccs_23,maccs_24,maccs_25,maccs_26,maccs_27,maccs_28,maccs_29,maccs_30,maccs_31,maccs_32,maccs_33,maccs_34,maccs_35,maccs_36,maccs_37,maccs_38,maccs_39,maccs_40,maccs_41,maccs_42,maccs_43,maccs_44,maccs_45,maccs_46,maccs_47,maccs_48,maccs_49,maccs_50,maccs_51,maccs_52,maccs_53,maccs_54,maccs_55,maccs_56,maccs_57,maccs_58,maccs_59,maccs_60,maccs_61,maccs_62,maccs_63,maccs_64,maccs_65,maccs_66,maccs_67,maccs_68,maccs_69,maccs_70,maccs_71,maccs_72,maccs_73,maccs_74,maccs_75,maccs_76,maccs_77,maccs_78,maccs_79,maccs_80,maccs_81,maccs_82,maccs_83,maccs_84,maccs_85,maccs_86,maccs_87,maccs_88,maccs_89,maccs_90,maccs_91,maccs_92,maccs_93,maccs_94,maccs_95,maccs_96,maccs_97,maccs_98,maccs_99,maccs_100,maccs_101,maccs_102,maccs_103,maccs_104,maccs_105,maccs_106,maccs_107,maccs_108,maccs_109,maccs_110,maccs_111,maccs_112,maccs_113,maccs_114,maccs_115,maccs_116,maccs_117,maccs_118,maccs_119,maccs_120,maccs_121,maccs_122,maccs_123,maccs_124,maccs_125,maccs_126,maccs_127,maccs_128,maccs_129,maccs_130,maccs_131,maccs_132,maccs_133,maccs_134,maccs_135,maccs_136,maccs_137,maccs_138,maccs_139,maccs_140,maccs_141,maccs_142,maccs_143,maccs_144,maccs_145,maccs_146,maccs_147,maccs_148,maccs_149,maccs_150,maccs_151,maccs_152,maccs_153,maccs_154,maccs_155,maccs_156,maccs_157,maccs_158,maccs_159,maccs_160,maccs_161,maccs_162,maccs_163,maccs_164,maccs_165,maccs_166,NR-AR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,1,0,0,0,1,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,1,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,1,1,0,0,0,0,1,1,1,0,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,0,0,1,1,1,0,0,1,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,1,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,1,1,0,1,1,1,0,0,0,1,0,1,1,0,1,0,1,1,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [14]:
# df_test 출력
df_test

,maccs_1,maccs_2,maccs_3,maccs_4,maccs_5,maccs_6,maccs_7,maccs_8,maccs_9,maccs_10,maccs_11,maccs_12,maccs_13,maccs_14,maccs_15,maccs_16,maccs_17,maccs_18,maccs_19,maccs_20,maccs_21,maccs_22,maccs_23,maccs_24,maccs_25,maccs_26,maccs_27,maccs_28,maccs_29,maccs_30,maccs_31,maccs_32,maccs_33,maccs_34,maccs_35,maccs_36,maccs_37,maccs_38,maccs_39,maccs_40,maccs_41,maccs_42,maccs_43,maccs_44,maccs_45,maccs_46,maccs_47,maccs_48,maccs_49,maccs_50,maccs_51,maccs_52,maccs_53,maccs_54,maccs_55,maccs_56,maccs_57,maccs_58,maccs_59,maccs_60,maccs_61,maccs_62,maccs_63,maccs_64,maccs_65,maccs_66,maccs_67,maccs_68,maccs_69,maccs_70,maccs_71,maccs_72,maccs_73,maccs_74,maccs_75,maccs_76,maccs_77,maccs_78,maccs_79,maccs_80,maccs_81,maccs_82,maccs_83,maccs_84,maccs_85,maccs_86,maccs_87,maccs_88,maccs_89,maccs_90,maccs_91,maccs_92,maccs_93,maccs_94,maccs_95,maccs_96,maccs_97,maccs_98,maccs_99,maccs_100,maccs_101,maccs_102,maccs_103,maccs_104,maccs_105,maccs_106,maccs_107,maccs_108,maccs_109,maccs_110,maccs_111,maccs_112,maccs_113,maccs_114,maccs_115,maccs_116,maccs_117,maccs_118,maccs_119,maccs_120,maccs_121,maccs_122,maccs_123,maccs_124,maccs_125,maccs_126,maccs_127,maccs_128,maccs_129,maccs_130,maccs_131,maccs_132,maccs_133,maccs_134,maccs_135,maccs_136,maccs_137,maccs_138,maccs_139,maccs_140,maccs_141,maccs_142,maccs_143,maccs_144,maccs_145,maccs_146,maccs_147,maccs_148,maccs_149,maccs_150,maccs_151,maccs_152,maccs_153,maccs_154,maccs_155,maccs_156,maccs_157,maccs_158,maccs_159,maccs_160,maccs_161,maccs_162,maccs_163,maccs_164,maccs_165,maccs_166,NR-AR
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,0,0,0,1,1,1,1,0,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,1,1,0,0,1,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,0,1,1,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [15]:
# df과 df_test에서 feature(X) 부분과 label(y) 부분 분리
X = df.iloc[:, :166]
y = df.iloc[:, 166]
X_test = df_test.iloc[:, :166]
y_test = df_test.iloc[:, 166]

In [16]:
# 반복 시행 시 같은 난수를 추출하기 위해 seed값 지정
seed = 42

## **1. Logistic Regression 모델**
### hyperparameter 튜닝

In [17]:
logistic_params_dict = {
	# 규제의 강도를 설정(1이 기본값)
	# 값이 작을수록 규제가 강해짐
	'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 7, 9, 11, 15, 20, 25, 30, 35, 40, 50, 100],
	
	# 규제의 종류를 설정
	# l1: Lasso 규제
	# l2: Ridge 규제
	'penalty': ['l1', 'l2'],

	# 최적화에 사용할 알고리즘 설정(lbfgs가 기본값)
	# liblinear: 작은 데이터셋에 적합한 최적화 알고리즘
	# saga: 대용량 데이터셋에 적합한 최적화 알고리즘
	'solver': ['liblinear', 'saga']
}

### **Logistic Regression 모델에서 위 hyperparameter로 만들 수 있는 가능한 모든 조합 만들기**

위에서 만든 logistic_params_dict가 올바른 형태인지 확인

In [18]:
# logistic_params_dict의 자료형이 딕셔너리인지 확인
if not isinstance(logistic_params_dict, dict):
	# logistic_params_dict가 딕셔너리가 아니면 에러 출력
	raise TypeError('Parameter grid is not a dict ({!r})'.format(logistic_params_dict))

if isinstance(logistic_params_dict, dict):
	# logistic_params_dict의 각 원소가 반복 가능(iterable)한지 확인
	# 여기서 반복 가능하다는 것은 리스트, 튜플, 세트, 문자열 등을 의미
	for key in logistic_params_dict:
		# logistic_params_dict가 딕셔너리가 아니면 에러 출력
		if not isinstance(logistic_params_dict[key], Iterable):
			raise TypeError('Parameter grid value is not iterable '
							'(key={!r}, value={!r})'.format(key, logistic_params_dict[key]))

In [19]:
# 딕셔너리의 key의 알파벳 순으로 정렬
items = sorted(logistic_params_dict.items())

# 다음과 같이 key는 key끼리, value는 value끼리 모아서 저장
# item 앞에 *를 붙이면 item의 iterable한 객체들을 분리하여 인자로 전달함
# ('C', 'penalty', 'solver')
# ([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 2, 3, 4, 5, 7, 9, 11, 15, 20, 25, 30, 35, 40, 50, 100], ['l1', 'l2'], ['liblinear', 'saga'])
keys, values = zip(*items)

# 조합된 parameter를 담을 list 선언
logistic_params_grid = []

# product 함수는 iterable한 객체의 모든 가능한 조합을 생성함
# value 앞에 *를 붙이면 value의 iterable한 객체들을 분리하여 인자로 전달함
# (0.1, 'l1', 'liblinear')처럼...
for v in product(*values):
	# {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} 이런 식으로 만들어서 하나씩 추가
	logistic_params_grid.append(dict(zip(keys, v))) 

In [20]:
# parameter 조합 출력
logistic_params_grid

[{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'},
 {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'},
 {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'},
 {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'},
 {'C': 0.2, 'penalty': 'l1', 'solver': 'liblinear'},
 {'C': 0.2, 'penalty': 'l1', 'solver': 'saga'},
 {'C': 0.2, 'penalty': 'l2', 'solver': 'liblinear'},
 {'C': 0.2, 'penalty': 'l2', 'solver': 'saga'},
 {'C': 0.3, 'penalty': 'l1', 'solver': 'liblinear'},
 {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'},
 {'C': 0.3, 'penalty': 'l2', 'solver': 'liblinear'},
 {'C': 0.3, 'penalty': 'l2', 'solver': 'saga'},
 {'C': 0.4, 'penalty': 'l1', 'solver': 'liblinear'},
 {'C': 0.4, 'penalty': 'l1', 'solver': 'saga'},
 {'C': 0.4, 'penalty': 'l2', 'solver': 'liblinear'},
 {'C': 0.4, 'penalty': 'l2', 'solver': 'saga'},
 {'C': 0.5, 'penalty': 'l1', 'solver': 'liblinear'},
 {'C': 0.5, 'penalty': 'l1', 'solver': 'saga'},
 {'C': 0.5, 'penalty': 'l2', 'solver': 'liblinear'},
 {'C': 0.5, 'penalty': 'l2', 'solver':

In [21]:
# parameter 조합 개수 출력
print(len(logistic_params_grid))

100


### **Logistic 평가 시작**

In [22]:
# 결과 딕셔너리 만들기
result = {}
result['model'] = {}
result['f1'] = {}

# result 출력해보기
result

{'model': {}, 'f1': {}}

#### Logistic의 parameter의 모든 경우의 수에 대해 f1_score 계산

In [23]:
# 모든 logistic_params_dict의 조합에 대해 반복
# tqdm은 프로그램 진행 상황을 시각적으로 보여줌
for p in tqdm(range(len(logistic_params_grid))):
	result['model']['model'+str(p)] = logistic_params_grid[p]
	result['f1']['model'+str(p)] = []
	
	# logistic_params_grid[p] 딕셔너리를 인자로 넘겨줄 때 ** 사용
	# **을 사용하면 딕셔너리의 키-값 쌍을 풀어서 인자로 전달 가능
	# ex. LogisticRegression(random_state=seed, C=1.0, penalty='l2', solver='lbfgs')
	model = LogisticRegression(random_state = seed, **logistic_params_grid[p])

	# Stratified K-fold 객체 생성
	skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
    
	# validation f1_score 저장할 리스트 생성
	val_f1 = []

	# k번 만큼 실험을 반복하며, 매 회차마다의 train 데이터를 다시 train과 validation data 범위의 index를 각각 train_index와 val_index에 저장
	for train_idx, val_idx in skf.split(X, y):
		# train data와 validation data 생성
		X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
		X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

		# 모델에 학습
		model.fit(X_train, y_train)

		# X_val 데이터로 예측한 결과 저장
		val_predict = model.predict(X_val)
	
		# val_predict의 f1_score를 하나씩 저장
		val_f1.append(f1_score(y_val, val_predict))
	
	# 각 모델 별 validation f1_score의 평균을 저장
	result['f1']['model'+str(p)].append(np.mean(val_f1))

100%|██████████| 100/100 [04:44<00:00,  2.85s/it]


In [24]:
# 결과 출력
result

{'model': {'model0': {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'},
  'model1': {'C': 0.1, 'penalty': 'l1', 'solver': 'saga'},
  'model2': {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'},
  'model3': {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'},
  'model4': {'C': 0.2, 'penalty': 'l1', 'solver': 'liblinear'},
  'model5': {'C': 0.2, 'penalty': 'l1', 'solver': 'saga'},
  'model6': {'C': 0.2, 'penalty': 'l2', 'solver': 'liblinear'},
  'model7': {'C': 0.2, 'penalty': 'l2', 'solver': 'saga'},
  'model8': {'C': 0.3, 'penalty': 'l1', 'solver': 'liblinear'},
  'model9': {'C': 0.3, 'penalty': 'l1', 'solver': 'saga'},
  'model10': {'C': 0.3, 'penalty': 'l2', 'solver': 'liblinear'},
  'model11': {'C': 0.3, 'penalty': 'l2', 'solver': 'saga'},
  'model12': {'C': 0.4, 'penalty': 'l1', 'solver': 'liblinear'},
  'model13': {'C': 0.4, 'penalty': 'l1', 'solver': 'saga'},
  'model14': {'C': 0.4, 'penalty': 'l2', 'solver': 'liblinear'},
  'model15': {'C': 0.4, 'penalty': 'l2', 'solver': 'saga'},
 

In [25]:
# best parameter 찾기

# 모델의 f1_score로만 list 생성
f1_list = list(map(lambda x: np.mean(x[1]), result['f1'].items()))
# f1_score가 최대인 곳의 index 저장
f1_max_idx = f1_list.index(max(f1_list))

# best parameter 저장
best_param = result['model'][f'model{f1_max_idx}']

In [26]:
# best_param이 위치한 index 찾기
m = list(result['model'].keys())[list(result['model'].values()).index(best_param)]

In [27]:
# val result 출력

# best parameter인 곳의 validation f1_score 저장
f1 = result['f1'][m]

# best parameter와 validation의 f1_score 출력
print(f"best param: {best_param} \
		\n[validation result] \
		\nf1: {f1[0]:.3f}")

best param: {'C': 15, 'penalty': 'l1', 'solver': 'liblinear'} 		
[validation result] 		
f1: 0.896


In [28]:
# test result 출력

# 최종으로 나온 best parameter의 Logistic Regression 모델
model = LogisticRegression(random_state = seed, **best_param)


# Stratified K-fold 객체 생성
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

# k번 만큼 실험을 반복하며, 매 회차마다의 train 데이터를 다시 train과 validation data 범위의 index를 각각 train_index와 val_index에 저장
for train_idx, val_idx in skf.split(X, y):
	# train data와 validation data 생성
	X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
	X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

	# tox21_train 데이터로 학습
	model.fit(X_train, y_train)

# tox21_test 데이터로 예측
pred = model.predict(X_test)
pred_score = model.predict_proba(X_test)[:, 1]

# best parameter와 예측 결과의 성능 지표 출력
print(f'[test result] \
		\nbest param: {best_param} \
		\nprecision: {precision_score(y_test, pred):.3f} \
		\nrecall: {recall_score(y_test, pred):.3f} \
		\naccuracy: {accuracy_score(y_test, pred):.3f} \
		\nauc: {roc_auc_score(y_test, pred_score):.3f} \
		\nf1: {f1_score(y_test, pred):.3f}')

[test result] 		
best param: {'C': 15, 'penalty': 'l1', 'solver': 'liblinear'} 		
precision: 0.166 		
recall: 0.516 		
accuracy: 0.869 		
auc: 0.699 		
f1: 0.251


# **2. MLP 모델 사용**
### **MLP hyperparameter 튜닝**

In [29]:
# MLP의 hyperparameter
mlp_params_dict = {
	# 모델에 있는 은닉층의 크기와 개수
	'hidden_layer_sizes': [(50), (100, 50, 10)],
	
	# 뉴런의 출력을 결정하는 활성화 함수 지정
	# relu: ReLU 함수, max(x, 0)과 동일
	# tanh: hyperbolic tangent 함수
	'activation': ['relu', 'tanh'],
	
	# 최적화에 사용될 알고리즘
	# adam: Adaptive Moment Estimation(Adam), 이전 그래디언트의 지수적인 이동 평균을 사용하여 학습률을 조절하는 방식으로 모델을 업데이트
	# sgd: Stochastic Gradient Descent(SGD), 데이터를 무작위로 선정하여 경사 하강법으로 매개변수를 갱신하는 방법
	'solver': ['adam', 'sgd'],
	
	# L2 정규화 항의 가중치
	'alpha': [0.0001, 0.001],
	
	# 학습률의 초기값 설정
	'learning_rate_init': [0.001, 0.01],
	
	# 최대 학습 과정 반복 횟수
	'max_iter': [50, 100]
}

### **MLP 모델에서 위 hyperparameter로 만들 수 있는 가능한 모든 조합 만들기**

위에서 만든 mlp_params_dict가 올바른 형태인지 확인

In [30]:
# mlp_params_dict의 자료형이 딕셔너리인지 확인
if not isinstance(mlp_params_dict, dict):
	# mlp_params_dict가 딕셔너리가 아니면 에러 출력
	raise TypeError('Parameter grid is not a dict ({!r})'.format(mlp_params_dict))

# mlp_params_dict가 dictionary라면
if isinstance(mlp_params_dict, dict):
	# mlp_params_dict의 각 원소가 반복 가능(iterable)한지 확인
	# 여기서 반복 가능하다는 것은 리스트, 튜플, 세트, 문자열 등을 의미
	for key in mlp_params_dict:
		# mlp_params_dict가 딕셔너리가 아니면 에러 출력
		if not isinstance(mlp_params_dict[key], Iterable):
			raise TypeError('Parameter grid value is not iterable '
							'(key={!r}, value={!r})'.format(key, mlp_params_dict[key]))

In [31]:
# 딕셔너리를 key의 알파벳 순으로 정렬
items = sorted(mlp_params_dict.items())

# 다음과 같이 key는 key끼리, value는 value끼리 모아서 저장
# item 앞에 *를 붙이면 item의 iterable한 객체들을 분리하여 인자로 전달함
# ('activation', 'alpha', 'hidden_layer_sizes', 'learning_rate_init', 'max_iter', 'solver')
# (['relu', 'tanh'], [0.0001, 0.001], [50, (100, 50, 10)], [0.001, 0.01], [50, 100], ['adam', 'sgd'])
keys, values = zip(*items)

# 조합된 parameter를 담을 list 선언
mlp_params_grid = []

# product 함수는 iterable한 객체의 모든 가능한 조합을 생성함
# value 앞에 *를 붙이면 value의 iterable한 객체들을 분리하여 인자로 전달함
# ('relu', 0.0001, 50, 0.001, 50, 'adam')처럼...
for v in product(*values):
	# {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': 50, 'learning_rate_init': 0.001, 'max_iter': 50, 'solver': 'adam'}
	# 이런 식으로 만들어서 하나씩 추가
	mlp_params_grid.append(dict(zip(keys, v))) 

In [32]:
# parameter 조합 출력
mlp_params_grid

[{'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.001,
  'max_iter': 50,
  'solver': 'adam'},
 {'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.001,
  'max_iter': 50,
  'solver': 'sgd'},
 {'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.001,
  'max_iter': 100,
  'solver': 'adam'},
 {'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.001,
  'max_iter': 100,
  'solver': 'sgd'},
 {'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.01,
  'max_iter': 50,
  'solver': 'adam'},
 {'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.01,
  'max_iter': 50,
  'solver': 'sgd'},
 {'activation': 'relu',
  'alpha': 0.0001,
  'hidden_layer_sizes': 50,
  'learning_rate_init': 0.01,
  'max_iter': 100,
  'solver': 'adam'},
 {'activation': 

In [33]:
# 가능한 parameter 조합 개수 출력
print(len(mlp_params_grid))

64


In [34]:
# 결과 딕셔너리 만들기
result = {}
result['model'] = {}
result['f1'] = {}

# result 출력해보기
result

{'model': {}, 'f1': {}}

In [35]:
# 모든 mlp_params_dict의 조합에 대해 반복
# tqdm은 프로그램 진행 상황을 시각적으로 보여줌
for p in tqdm(range(len(mlp_params_grid))):
	# 현재 적용한 parameter 조합을 저장
	result['model']['model'+str(p)] = mlp_params_grid[p]
	# f1_score를 저장할 리스트 생성
	result['f1']['model'+str(p)] = []

	# mlp_params_grid[p] 딕셔너리를 인자로 넘겨줄 때 ** 사용
	# **을 사용하면 딕셔너리의 키-값 쌍을 풀어서 인자로 전달 가능
	# ex. MLPClassifier(random_state=seed, activation='relu', alpha=0.0001, hidden_layer_sizes=50, learning_rate_init=0.001, max_iter=50, solver='adam')
	model = MLPClassifier(random_state = seed, **mlp_params_grid[p])

	# Stratified K-fold 객체 생성
	skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

	# validation f1_score 저장할 리스트 생성
	val_f1 = []

	# k번 만큼 실험을 반복하며, 매 회차마다의 train 데이터를 다시 train과 validation data 범위의 index를 각각 train_index와 val_index에 저장
	for train_idx, val_idx in skf.split(X, y):
		# train data와 validation data 생성
		X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
		X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

		# 모델에 학습
		model.fit(X_train, y_train)

		# X_train 데이터로 예측한 결과 저장
		train_predict = model.predict(X_train)

		# X_val 데이터로 예측한 결과 저장
		val_predict = model.predict(X_val)
		
		# val_predict의 f1_score를 하나씩 저장
		val_f1.append(f1_score(y_val, val_predict))

	# 각 모델 별 validation f1_score의 평균을 저장
	result['f1']['model'+str(p)].append(np.mean(val_f1))

100%|██████████| 64/64 [22:11<00:00, 20.80s/it]


In [36]:
# 결과 출력
result

{'model': {'model0': {'activation': 'relu',
   'alpha': 0.0001,
   'hidden_layer_sizes': 50,
   'learning_rate_init': 0.001,
   'max_iter': 50,
   'solver': 'adam'},
  'model1': {'activation': 'relu',
   'alpha': 0.0001,
   'hidden_layer_sizes': 50,
   'learning_rate_init': 0.001,
   'max_iter': 50,
   'solver': 'sgd'},
  'model2': {'activation': 'relu',
   'alpha': 0.0001,
   'hidden_layer_sizes': 50,
   'learning_rate_init': 0.001,
   'max_iter': 100,
   'solver': 'adam'},
  'model3': {'activation': 'relu',
   'alpha': 0.0001,
   'hidden_layer_sizes': 50,
   'learning_rate_init': 0.001,
   'max_iter': 100,
   'solver': 'sgd'},
  'model4': {'activation': 'relu',
   'alpha': 0.0001,
   'hidden_layer_sizes': 50,
   'learning_rate_init': 0.01,
   'max_iter': 50,
   'solver': 'adam'},
  'model5': {'activation': 'relu',
   'alpha': 0.0001,
   'hidden_layer_sizes': 50,
   'learning_rate_init': 0.01,
   'max_iter': 50,
   'solver': 'sgd'},
  'model6': {'activation': 'relu',
   'alpha': 0.000

In [37]:
# best parameter 찾기

# 모델의 f1_score로만 list 생성
f1_list = list(map(lambda x: np.mean(x[1]), result['f1'].items()))
# f1_score가 최대인 곳의 index 저장
f1_max_idx = f1_list.index(max(f1_list))

# best parameter 저장
best_param = result['model'][f'model{f1_max_idx}']

In [38]:
# best_param이 위치한 index 찾기
m = list(result['model'].keys())[list(result['model'].values()).index(best_param)]

In [39]:
# val result 출력

# best parameter인 곳의 validation f1_score 저장
f1 = result['f1'][m]

# best parameter와 validation의 f1_score 출력
print(f"best param: {best_param} \
		\n[validation result] \
		\nf1: {f1[0]:.3f}")

best param: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (100, 50, 10), 'learning_rate_init': 0.01, 'max_iter': 100, 'solver': 'adam'} 		
[validation result] 		
f1: 0.974


In [40]:
# test result 출력

# 최종으로 나온 best parameter의 MLP 모델
model = MLPClassifier(random_state = seed, **best_param)

# Stratified K-fold 객체 생성
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

# k번 만큼 실험을 반복하며, 매 회차마다의 train 데이터를 다시 train과 validation data 범위의 index를 각각 train_index와 val_index에 저장
for train_idx, val_idx in skf.split(X, y):
	# train data와 validation data 생성
	X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
	X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]

	# tox21_train 데이터로 학습
	model.fit(X_train, y_train)

# tox21_test 데이터로 예측
pred = model.predict(X_test)
pred_score = model.predict_proba(X_test)[:, 1]

# best parameter와 예측 결과의 성능 지표 출력
print(f'[test result] \
		\nbest param: {best_param} \
		\nprecision: {precision_score(y_test, pred):.3f} \
		\nrecall: {recall_score(y_test, pred):.3f} \
		\naccuracy: {accuracy_score(y_test, pred):.3f} \
		\nauc: {roc_auc_score(y_test, pred_score):.3f} \
		\nf1: {f1_score(y_test, pred):.3f}')

[test result] 		
best param: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (100, 50, 10), 'learning_rate_init': 0.01, 'max_iter': 100, 'solver': 'adam'} 		
precision: 0.409 		
recall: 0.435 		
accuracy: 0.949 		
auc: 0.744 		
f1: 0.422


## **3. 최종 결과 비교**

Logistic Regression 모델을 사용한 결과 Precision은 0.166, Recall은 0.516, Accuracy는 0.869, Auc은 0.699, F1 score는 0.251이 나왔다.

MLP 모델을 사용한 결과 Precision은 0.409, Recall은 0.435, Accuracy는 0.949, Auc은 0.744, F1 score는 0.422이 나왔다.